In [14]:
from maze import Maze
from robot import Robot
from take_n_steps import take_n_steps
from print_maze import print_maze
import numpy as np
import sys


# global dictionaries for robot movement and sensing
dir_sensors = {'u': ['l', 'u', 'r'], 'r': ['u', 'r', 'd'],
               'd': ['r', 'd', 'l'], 'l': ['d', 'l', 'u'],
               'up': ['l', 'u', 'r'], 'right': ['u', 'r', 'd'],
               'down': ['r', 'd', 'l'], 'left': ['d', 'l', 'u']}
dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
            'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}
dir_reverse = {'u': 'd', 'r': 'l', 'd': 'u', 'l': 'r',
               'up': 'd', 'right': 'l', 'down': 'u', 'left': 'r'}

# Create a maze
testmaze = Maze("test_maze_02.txt")

In [92]:
# Intitialize a robot; robot receives info about maze dimensions.
testrobot = Robot(testmaze.dim, (0,0), 'u')

# Set the robot in the start position. Note that robot position
# parameters are independent of the robot itself.
robot_pos = {'location': [0, 0], 'heading': 'u'}

cells = [["   " for x in range(testmaze.dim)] for row in range(testmaze.dim)]     

# Take N steps
hit_goal = False

while testrobot.move_count < 1000 and not hit_goal:
    robot_pos = take_n_steps(1, testmaze, testrobot, robot_pos)

    goal_bounds = [testmaze.dim/2 - 1, testmaze.dim/2]
    if robot_pos['location'][0] in goal_bounds and robot_pos['location'][1] in goal_bounds:
        hit_goal = True

#cells = [["   " for x in range(testmaze.dim)] for row in range(testmaze.dim)]     
#cells[testrobot.current_location[0]][testrobot.current_location[1]] = "XXX"
#print_maze(testmaze,cells)

import get_cells as gc
cells = gc.get_visited_cells(testrobot,testmaze)
cells[testrobot.current_location[0]][testrobot.current_location[1]] = "XXX"
print testrobot.move_count
print_maze(testmaze,cells)

242
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| X   X   X   X   X   X   X   X | X   X | X   X   X   X |
+---+---+---        +---+---                +---+---    +
| X   X   X | X | X   X   X   X   X | X   X |           |
    +---        +---+---+---                            +
| X   X | X   X |     X   X | X | X | X | X     |   |   |
                +---                        +---+---    +
| X | X   X | X | X   X | X   X | X   X | X | X       X |
    +---+---        +---                        +---    +
| X   X   X |   | X   X   X |   | X | X   X | X   X     |
    +---    +---        +---+---    +---+---+---    +---+
| X   X   X   X   X | X   X   X   X   X     | X   X   X |
        +---            +---+---+---+---+---    +---    +
|   | X   X   X | X   X |XXX      X |       | X   X | X |
+---+---+---    +---+---                                +
| X   X   X   X | X   X |       |       |   | X | X | X |
    +---+---            +---+---    +---                +
|   | X   

In [93]:
testrobot.next_locations_table[(1,8)]

[(2, 8), (3, 8), (4, 8), (1, 7), (1, 6), (1, 5), (0, 8)]

In [94]:
start = (0,0)
goal = testrobot.current_location
maze_dim = testrobot.maze_dim

next_locations = testrobot.next_locations_table
visited_locations = []
for loc in testrobot.next_locations_table.keys():
    if loc:
        visited_locations.append(loc)
visited_locations.append(goal)

heuristic_cost_estimate = np.zeros((maze_dim,maze_dim))
for x in [i for i in range(-maze_dim/2,maze_dim/2+1) if i != 0]:
    for y in [j for j in range(-maze_dim/2,maze_dim/2+1) if j != 0]:
        if x < 0: 
            dx = maze_dim/2 
        else: 
            dx = maze_dim/2 - 1
        if y < 0:
            dy = maze_dim/2
        else:
            dy = maze_dim/2 - 1
        heuristic_cost_estimate[x+dx][y+dy] = (abs(x)+abs(y))*2 - 4

In [95]:
def a_star(start,goal):
    closedSet = []
    openSet = [start]
    cameFrom = dict()
    
    gScore = {x:maze_dim**2 for x in visited_locations}
    gScore[start] = 0
    fScore = {x:maze_dim**2 for x in visited_locations}
    fScore[start] = heuristic_cost_estimate[start]
    
    def get_min_fscore(locations):
        location_fScore = [fScore[x] for x in locations]
        min_score = min(location_fScore)
        return locations[location_fScore.index(min_score)]
    
    def dist_between(loc1,loc2):
        loc1 = np.array(loc1)
        loc2 = np.array(loc2)
        return sum(abs(loc2-loc1))
    
    while openSet:
        current = get_min_fscore(openSet)
        if current == goal:
            return reconstruct_path(cameFrom,current)
        openSet.remove(current)
        closedSet.append(current)
        
        for next_location in next_locations[current]:
            if next_location in closedSet:
                continue
            
            if next_location not in openSet:
                openSet.append(next_location)
            
            tentative_gScore = gScore[current] + dist_between(current,next_location)
            if tentative_gScore >= gScore[next_location]:
                continue
            
            cameFrom[next_location] = current
            gScore[next_location] = tentative_gScore
            fScore[next_location] = gScore[next_location] + heuristic_cost_estimate[next_location]
            
    print "Failed"

def reconstruct_path(cameFrom,current):
    total_path = [current]
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.append(current)
    return total_path

In [96]:
best_path = a_star(start,goal)

In [97]:
import get_cells as gc
cells = gc.get_visited_cells(testrobot,testmaze)
for x in best_path:
    cells[x[0]][x[1]] = " 0 "

print "steps exploring = " + str(testrobot.move_count)
print "best path length = " + str(len(best_path))
print "total score = " + str(testrobot.move_count/30 + len(best_path))
print_maze(testmaze,cells)

steps exploring = 242
best path length = 29
total score = 37
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| 0   X   X   0   X   X   0   0 | 0   0 | 0   X   X   0 |
+---+---+---        +---+---                +---+---    +
| X   X   X | X | X   X   X   0   0 | 0   0 |           |
    +---        +---+---+---                            +
| X   X | X   X |     X   X | X | X | X | X     |   |   |
                +---                        +---+---    +
| X | X   X | X | X   X | X   X | X   X | X | 0       0 |
    +---+---        +---                        +---    +
| X   X   X |   | X   X   X |   | X | X   X | 0   0     |
    +---    +---        +---+---    +---+---+---    +---+
| X   X   X   X   X | X   X   X   X   X     | X   0   0 |
        +---            +---+---+---+---+---    +---    +
|   | X   X   X | X   X | 0       0 |       | X   X | X |
+---+---+---    +---+---                                +
| X   X   X   X | X   X |       |       |   | X | X | X |
    +---+--

In [80]:
print(best_path)

[(6, 7), (6, 8), (8, 8), (8, 9), (10, 9), (10, 11), (9, 11), (9, 12), (11, 12), (11, 13), (8, 13), (5, 13), (5, 12), (4, 12), (4, 13), (1, 13), (1, 11), (2, 11), (2, 10), (4, 10), (4, 9), (3, 9), (3, 8), (2, 8), (1, 8), (1, 7), (0, 7), (0, 6), (0, 3), (0, 0)]


In [84]:
next_locations[(1,8)]

[(1, 7), (1, 6), (1, 5), (0, 8), (2, 8)]

In [85]:
next_locations[(2,8)]

[(1, 8), (0, 8), (3, 8)]

In [74]:
print(heuristic_cost_estimate)

[[ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.  18.  20.  22.  24.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.

In [106]:
def a_star(start,goal):
    closedSet = []
    openSet = [start]
    cameFrom = dict()
    
    gScore = {x:maze_dim**2 for x in visited_locations}
    gScore[start] = 0
    fScore = {x:maze_dim**2 for x in visited_locations}
    fScore[start] = heuristic_cost_estimate[start]
    
    def get_min_fscore(locations):
        location_fScore = [fScore[x] for x in locations]
        min_score = min(location_fScore)
        return locations[location_fScore.index(min_score)]
    
    def dist_between(loc1,loc2):
        loc1 = np.array(loc1)
        loc2 = np.array(loc2)
        return sum(abs(loc2-loc1))
    
    while openSet:
    #for x in range(1):
        current = get_min_fscore(openSet)
        if current == goal:
            return reconstruct_path(cameFrom,current), cameFrom
        openSet.remove(current)
        closedSet.append(current)
        
        for next_location in next_locations[current]:
            #prevent openSet from looping back on itself
            if next_location in closedSet:
                continue
            
            #add next_locations to openSet
            if next_location not in openSet:
                openSet.append(next_location)
            
            tentative_gScore = gScore[current] + dist_between(current,next_location)
            if tentative_gScore >= gScore[next_location]:
                continue
            
            cameFrom[next_location] = current
            gScore[next_location] = tentative_gScore
            fScore[next_location] = gScore[next_location] + heuristic_cost_estimate[next_location]
            
    print "Failed"

def reconstruct_path(cameFrom,current):
    total_path = [current]
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.append(current)
    return total_path

In [107]:
best_path,cameFrom = a_star(start,goal)

In [72]:
print(heuristic_cost_estimate)

[[ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.  18.  20.  22.  24.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.